# GEO proximity-analysis Ejercicio

# Introducción

Usted es parte de un equipo de respuesta a crisis y desea identificar cómo los hospitales han respondido a las colisiones en la ciudad de Nueva York.

<centro>
<img src = "https://i.imgur.com/wamd0n7.png" width = "450"> <br/>
</center>

Antes de comenzar, ejecute la celda de código a continuación para configurar todo.

In [1]:
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon

import folium
from folium import Choropleth, Marker
from folium.plugins import HeatMap, MarkerCluster

In [2]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# Ejercicios

### 1) Visualice los datos de colisión.

Ejecute la celda de código a continuación para cargar una 'colisión' GeoDataFrame que rastrea las principales colisiones de vehículos motorizados en 2013-2018.

In [4]:
collisions = gpd.read_file("data/GEO/NYPD_Motor_Vehicle_Collisions.shp")
collisions.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET,CROSS STRE,OFF STREET,...,CONTRIBU_2,CONTRIBU_3,CONTRIBU_4,UNIQUE KEY,VEHICLE TY,VEHICLE _1,VEHICLE _2,VEHICLE _3,VEHICLE _4,geometry
0,07/30/2019,0:00,BRONX,10464,40.841100,-73.784960,"(40.8411, -73.78496)",None,None,121 PILOT STREET,...,Unspecified,None,None,4180045,Sedan,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,None,None,POINT (1043750.211 245785.815)
1,07/30/2019,0:10,QUEENS,11423,40.710827,-73.770660,"(40.710827, -73.77066)",JAMAICA AVENUE,188 STREET,None,...,None,None,None,4180007,Sedan,Sedan,None,None,None,POINT (1047831.185 198333.171)
2,07/30/2019,0:25,None,None,40.880318,-73.841286,"(40.880318, -73.841286)",BOSTON ROAD,None,None,...,None,None,None,4179575,Sedan,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (1028139.293 260041.178)
3,07/30/2019,0:35,MANHATTAN,10036,40.756744,-73.984590,"(40.756744, -73.98459)",None,None,155 WEST 44 STREET,...,None,None,None,4179544,Box Truck,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (988519.261 214979.320)
4,07/30/2019,10:00,BROOKLYN,11223,40.600090,-73.965910,"(40.60009, -73.96591)",AVENUE T,OCEAN PARKWAY,None,...,None,None,None,4180660,Station Wagon/Sport Utility Vehicle,Bike,None,None,None,POINT (993716.669 157907.212)


Use las columnas "LATITUD" y "LONGITUD" para crear un mapa interactivo para visualizar los datos de colisión. ¿Qué tipo de mapa crees que es más efectivo?

In [5]:
m_1 = folium.Map(location=[40.7, -74], zoom_start=11) 

# Visualize the collision data
HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m_1)

# Show the map
embed_map(m_1, "q_1.html")

### 2) Comprender la cobertura hospitalaria.

Ejecute la siguiente celda de código para cargar los datos del hospital.

In [6]:
hospitals = gpd.read_file("data/nyu_2451_34494.shp")
hospitals.head()

,id,name,address,zip,factype,facname,capacity,capname,bcode,xcoord,ycoord,latitude,longitude,geometry
0,317000001H1178,BRONX-LEBANON HOSPITAL CENTER - CONCOURSE DIVI...,1650 Grand Concourse,10457,3102,Hospital,415,Beds,36005,1008872.0,246596.0,40.843490,-73.911010,POINT (1008872.000 246596.000)
1,317000001H1164,BRONX-LEBANON HOSPITAL CENTER - FULTON DIVISION,1276 Fulton Ave,10456,3102,Hospital,164,Beds,36005,1011044.0,242204.0,40.831429,-73.903178,POINT (1011044.000 242204.000)
2,317000011H1175,CALVARY HOSPITAL INC,1740-70 Eastchester Rd,10461,3102,Hospital,225,Beds,36005,1027505.0,248287.0,40.848060,-73.843656,POINT (1027505.000 248287.000)
3,317000002H1165,JACOBI MEDICAL CENTER,1400 Pelham Pkwy,10461,3102,Hospital,457,Beds,36005,1027042.0,251065.0,40.855687,-73.845311,POINT (1027042.000 251065.000)
4,317000008H1172,LINCOLN MEDICAL & MENTAL HEALTH CENTER,234 E 149 St,10451,3102,Hospital,362,Beds,36005,1005154.0,236853.0,40.816758,-73.924478,POINT (1005154.000 236853.000)


Use las columnas "latitud" y "longitud" para visualizar las ubicaciones de los hospitales.

In [8]:
m_2 = folium.Map(location=[40.7, -74], zoom_start=11) 

for idx, row in hospitals.iterrows():
    Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(m_2)
        
# Show the map
embed_map(m_2, "q_2.html")

### 3) ¿Cuándo estaba el hospital más cercano a más de 10 kilómetros de distancia?

Cree un DataFrame `outside_range` que contenga todas las filas de` colisiones` con accidentes que ocurrieron a más de 10 kilómetros del hospital más cercano.

Tenga en cuenta que tanto "hospitales" como "colisiones" tienen EPSG 2263 como sistema de referencia de coordenadas, y EPSG 2263 tiene unidades de medidores.

In [9]:
coverage = gpd.GeoDataFrame(geometry=hospitals.geometry).buffer(10000)
my_union = coverage.geometry.unary_union
outside_range = collisions.loc[~collisions["geometry"].apply(lambda x: my_union.contains(x))]

La siguiente celda de código calcula el porcentaje de colisiones que ocurrieron a más de 10 kilómetros del hospital más cercano.

In [10]:
percentage = round(100*len(outside_range)/len(collisions), 2)
print("Percentage of collisions more than 10 km away from the closest hospital: {}%".format(percentage))

Percentage of collisions more than 10 km away from the closest hospital: 15.12%


### 4) Haga un recomendador.

Cuando se producen colisiones en lugares distantes, se vuelve aún más vital que las personas heridas sean transportadas al hospital disponible más cercano.

Con esto en mente, decide crear un recomendación que:
- toma la ubicación del bloqueo (en EPSG 2263) como entrada,
- encuentra el hospital más cercano (donde los cálculos de distancia se realizan en EPSG 2263), y
- devuelve el nombre del hospital más cercano.

In [11]:
def best_hospital(collision_location):
    idx_min = hospitals.geometry.distance(collision_location).idxmin()
    my_hospital = hospitals.iloc[idx_min]
    name = my_hospital["name"]
    latitude = my_hospital["latitude"]
    longitude = my_hospital["longitude"]
    return pd.Series({'name': name, 'lat': latitude, 'long': longitude})
    return name

# Test your function: this should suggest CALVARY HOSPITAL INC
print(best_hospital(outside_range.geometry.iloc[0]))

name    CALVARY HOSPITAL INC
lat                  40.8481
long                -73.8437
dtype: object


### 5) ¿Qué hospital tiene la mayor demanda?

Considerando solo las colisiones en el marco de datos `outside_range`, ¿qué hospital es el más recomendado?

Su respuesta debe ser una cadena de Python que coincida exactamente con el nombre del hospital devuelto por la función que creó en ** 4) **.

In [13]:
highest_demand = outside_range.geometry.apply(best_hospital).name.value_counts().idxmax()

### 6) ¿Dónde debería la ciudad construir nuevos hospitales?

Ejecute la siguiente celda de código (sin cambios) para visualizar las ubicaciones de los hospitales, además de las colisiones que ocurrieron a más de 10 kilómetros del hospital más cercano.

In [14]:
m_6 = folium.Map(location=[40.7, -74], zoom_start=11) 

folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m_6)
HeatMap(data=outside_range[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m_6)
folium.LatLngPopup().add_to(m_6)

embed_map(m_6, 'm_6.html')

Haga clic en cualquier parte del mapa para ver una ventana emergente con la ubicación correspondiente en latitud y longitud.

La ciudad de Nueva York le solicita ayuda para decidir las ubicaciones de dos nuevos hospitales. Desean específicamente su ayuda con la identificación de ubicaciones para llevar el porcentaje calculado del paso ** 3) ** a menos del diez por ciento. Usando el mapa (y sin preocuparse por las leyes de zonificación o qué edificios potenciales tendrían que ser removidos para construir los hospitales), ¿puede identificar dos ubicaciones que ayudarían a la ciudad a lograr este objetivo?

Ponga la latitud y longitud propuestas para el hospital 1 en `lat_1` y` long_1`, respectivamente. (Del mismo modo para el hospital 2.)

Luego, ejecute el resto de la celda como está para ver el efecto de los nuevos hospitales. Su respuesta se marcará como correcta, si los dos nuevos hospitales reducen el porcentaje a menos del diez por ciento.

In [18]:
# Proposed location of hospital 1
lat_1 = 40.6714
long_1 = -73.8492

# Proposed location of hospital 2
lat_2 = 40.6702
long_2 = -73.7612

# Do not modify the code below this line
try:
    new_df = pd.DataFrame(
        {'Latitude': [lat_1, lat_2],
         'Longitude': [long_1, long_2]})
    new_gdf = gpd.GeoDataFrame(new_df, geometry=gpd.points_from_xy(new_df.Longitude, new_df.Latitude))
    new_gdf.crs = {'init' :'epsg:4326'}
    new_gdf = new_gdf.to_crs(epsg=2263)
    # get new percentage
    new_coverage = gpd.GeoDataFrame(geometry=new_gdf.geometry).buffer(10000)
    new_my_union = new_coverage.geometry.unary_union
    new_outside_range = outside_range.loc[~outside_range["geometry"].apply(lambda x: new_my_union.contains(x))]
    new_percentage = round(100*len(new_outside_range)/len(collisions), 2)
    print("(NEW) Percentage of collisions more than 10 km away from the closest hospital: {}%".format(new_percentage))
    # make the map
    m = folium.Map(location=[40.7, -74], zoom_start=11) 
    folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m)
    folium.GeoJson(new_coverage.geometry.to_crs(epsg=4326)).add_to(m)
    for idx, row in new_gdf.iterrows():
        Marker([row['Latitude'], row['Longitude']]).add_to(m)
    HeatMap(data=new_outside_range[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m)
    folium.LatLngPopup().add_to(m)
    display(embed_map(m, 'q_6.html'))
except:
    q_6.hint()

(NEW) Percentage of collisions more than 10 km away from the closest hospital: 9.12%
